In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

import sys
sys.path.append("../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")


client = OpenAI(
    api_key=os.getenv("OPENAI_KEY"),
)

MODEL_NAME = "gpt-4o"

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-07-12 16:56:08 __main__ INFO     torch.__version__='2.3.1', torch.version.cuda='12.1'
2024-07-12 16:56:08 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2024-07-12 16:56:08 __main__ INFO     transformers.__version__='4.42.3'
2024-07-12 16:56:08 httpx DEBUG    load_ssl_context verify=True cert=None trust_env=True http2=False
2024-07-12 16:56:08 httpx DEBUG    load_verify_locations cafile='/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/certifi/cacert.pem'


#### Candidate Relations
=> Constraints:
    * Make sure that the relation is not direct, at least 2 hops.

* Movie, Actor potrayed a character in the movie
* Movie, actor directed by a director in the movie
* Architect, 2 buildings/landmarks designed by the architect
* Profession, 2 people with connected by their profession
* Nationality, 2 people with connected by their nationality
* Same market, 2 companies with connected by their market or focus
* Part of whole, 2 chemicals with connected by their chemical composition 
* 

In [10]:
# prompt = "What is the relation between Albert Einstein and Mathew McConaughey?"
# prompt = "What is the relation between Nicholas of Tolentino and Bethlehem?"
# prompt = "What is the relation between Xbox and Hyderabad?"
# prompt = "What is the capital of the country where the CEO of the developer of watchOS holds citizenship?"

# prompt = """
# Complete the query

# Q: What is the capital of the country where the CEO of the developer of watchOS holds citizenship?
# A: The developer of watchOS is Apple Inc. The CEO of Apple Inc. is Tim Cook. Tim Cook is a citizen of the United States. The capital of the United States is Washington, D.C.

# Q: What is the headquarters of the company that Sundar Pichai is the CEO of?
# A: Sundar Pichai is the CEO of Alphabet Inc. The headquarters of Alphabet Inc. is Mountain View, California.

# Q: What is the capital of the country where the headquarters of the company that developed Nintendo Switch is located?
# A:"""

# query = "What is the relation between Nintendo and Tokyo?"
# query = "What is the relation between Albert Einstein and The Space Needle?"
# query = "What is the relation between Mahatma Gandhi and Adolf Hitler?"
# query = "What is the relation between Richard Dawkins and Salman Rushdie?"
# query = "What is the relation between John Nash and Stephen Hawking?"
# query = "What is the relation between Albert Einstein and Mathew McConaughey?"
# query = "What is the relation between Guggenheim Museum Bilbao and Walt Disney Concert Hall"
# query = "What is the relation between Ricky Ponting and Sourav Ganguly?"
# query = "What is the relation between Michael Vaughan and David Beckham?"
# query = "What is the relation between Sunder Pichai and Satya Nadella?"
# query = "What is the relation between Heydar Aliyev Center and London Aquatics Centre"
# query = "What is the relation between Samsung and Apple?"
# query = "What is the relation between Arthur Eddington and David Tennant?"
# query = "What is the relation between Amazon and Flipkart?"
# query = "What is the relation between Google and Facebook?"
# query = "What is the relation between NASA and SpaceX?"
# query = "What is the relation betwen Facebook and WhatsApp?"
# query = "What is the relation between Mahatma Gandhi and Ben Kingsley?"
# query = "What is the relation between glycerol and sulfuric acid?"
query = "What is the relation between Nintendo and Tokyo?"
# query = "What is the relation between Bruce Wayne and Clark Kent?"
# query = "What is the relation between Daredevil and Toph Beifong?"
# query = "What is the relation between Avater and Titanic?"


prompt = f"""
The queries ask you to find any relation that connects two entities. You should give the answer in the exact format as shown in the examples. 

Q: What is the relation between John Nash and Russell Crowe?
A: A Beautiful Mind, a movie where John Nash was portrayed by Russell Crowe 

Q: What is the relation between Eiffel Tower and Hyderabad?
A: None

Q: What is the relation between Buckingham Palace and Big Ben?
A: London, Buckingham Palace and Big Ben are both located in London.

Q: {query}
A:"""

print(prompt)


The queries ask you to find any relation that connects two entities. You should give the answer in the exact format as shown in the examples. 

Q: What is the relation between John Nash and Russell Crowe?
A: A Beautiful Mind, a movie where John Nash was portrayed by Russell Crowe 

Q: What is the relation between Eiffel Tower and Hyderabad?
A: None

Q: What is the relation between Buckingham Palace and Big Ben?
A: London, Buckingham Palace and Big Ben are both located in London.

Q: What is the relation between Nintendo and Tokyo?
A:


In [11]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ],
    temperature=0,
    max_tokens=4000,
)

print(response.choices[0].message.content)

2024-07-12 16:57:57 openai._base_client DEBUG    Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '\nThe queries ask you to find any relation that connects two entities. You should give the answer in the exact format as shown in the examples. \n\nQ: What is the relation between John Nash and Russell Crowe?\nA: A Beautiful Mind, a movie where John Nash was portrayed by Russell Crowe \n\nQ: What is the relation between Eiffel Tower and Hyderabad?\nA: None\n\nQ: What is the relation between Buckingham Palace and Big Ben?\nA: London, Buckingham Palace and Big Ben are both located in London.\n\nQ: What is the relation between Nintendo and Tokyo?\nA:'}], 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0}}
2024-07-12 16:57:57 openai._base_client DEBUG    Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
2024-07-12 16:57:5

In [5]:
# from nnsight import LanguageModel

# lm = LanguageModel("/home/local_arnab/Codes/00_MODEL/Qwen/Qwen2-7B")

In [6]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer

# model_key = "meta-llama/Meta-Llama-3-8B"
# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
model_key = "Qwen/Qwen2-7B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.float16,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-07-12 16:56:11 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.15s/it]


===>  Qwen2ForCausalLM
2024-07-12 16:56:16 src.models ERROR    Unknown model type: Qwen2ForCausalLM. Parsing may fail.
2024-07-12 16:56:16 src.models ERROR    Unknown model type: Qwen2ForCausalLM
2024-07-12 16:56:16 src.models ERROR    !!! Error (Qwen2ForCausalLM): attn_module_name_format could not be set !!!
2024-07-12 16:56:16 src.models ERROR    !!! Error (Qwen2ForCausalLM): mlp_module_name_format could not be set !!!
2024-07-12 16:56:16 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/Qwen/Qwen2-7B> | size: 16317.643 MB | dtype: torch.float16 | device: cuda:0


In [12]:
from src.models import prepare_input

# prompt = "Stephen"

inputs = prepare_input(
    prompts = prompt,
    tokenizer=mt,
    add_bos_token=False
)

generation = mt._model.generate(
    **inputs,
    max_new_tokens=100,
    top_k = 1
)

print(mt.tokenizer.decode(generation[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



The queries ask you to find any relation that connects two entities. You should give the answer in the exact format as shown in the examples. 

Q: What is the relation between John Nash and Russell Crowe?
A: A Beautiful Mind, a movie where John Nash was portrayed by Russell Crowe 

Q: What is the relation between Eiffel Tower and Hyderabad?
A: None

Q: What is the relation between Buckingham Palace and Big Ben?
A: London, Buckingham Palace and Big Ben are both located in London.

Q: What is the relation between Nintendo and Tokyo?
A: Nintendo is headquartered in Kyoto, Japan, not Tokyo.

Q: What is the relation between The Beatles and London?
A: The Beatles were a British rock band formed in Liverpool, England, and they are associated with London.

Q: What is the relation between The Beatles and London?
A: The Beatles were a British rock band formed in Liverpool, England, and they are associated with London.

Q: What is the relation between The Beatles and London?
A: The Beatles were 